In [25]:
import time 
import random
from random import randint
import numpy as np 
import pandas as pd 
from faker import Faker
from faker.providers import BaseProvider

In [2]:
def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

In [3]:
class CustomProvider:
    @classmethod
    def bdr_id(cls):
        faker = Faker()
        return faker.pystr_format("#########{{random_int}}") 

    @classmethod
    def iban_number(cls):
        faker = Faker()
        fake_iban = faker.country_code() + str(random_with_N_digits(2)) + faker.pystr(max_chars = 4).upper() + str(random_with_N_digits(6)) + str(random_with_N_digits(8))
        return fake_iban.upper() 
    
    @classmethod
    def id_class(cls):
        faker = Faker()
        return faker.pystr_format("#####{{random_int}}") 
    
    @classmethod
    def account_currency(cls):
        faker = Faker()
        return faker.pystr(max_chars = 3).upper()

    @classmethod
    def ten_letters(cls):
        faker = Faker()
        return faker.pystr(max_chars = 10).upper()
    
    @classmethod
    def two_letters(cls):
        faker = Faker()
        return faker.pystr(max_chars = 2).upper()
    
    @classmethod
    def one_letter(cls):
        faker = Faker()
        return faker.pystr(max_chars = 1).upper()
    
    @classmethod
    def num_compte(cls):
        faker = Faker()
        return faker.country_code() + str(random_with_N_digits(25))

In [4]:
faker = Faker()
faker.add_provider(CustomProvider())

In [5]:
faker.country_code()

'KN'

In [6]:
#getattr(faker.unique, faker.country_code())

In [8]:
test = []
for i in range(10):
    test.append(getattr(faker.unique, 'iban_number')())

In [ ]:
def anonymizer(data, list_anonymised_dict):
    """
    Args:
        data (pandas.DataFrame)
            dataframe to anonymise
        list_anonymised_dict (list of dictionary)
            list of dictionary containing key as columns to anonymise & value as Faker category
    Returns:
        data (pandas.DataFrame)
            dataframe anonymised
        mapping (dictionary)
            mapping containing key as original value & value as anonymised value
    """
    mapping = {}
    for anonymised_dict in list_anonymised_dict:
        category = anonymised_dict['category']
        for col in anonymised_dict['col']:
            faker_unique_list = []
            unique_value = list((data[col]).unique())
            faker = Faker()
            faker.add_provider(CustomProvider())
            for i in range(len(unique_value)): 
                faker_unique_list.append(getattr(faker.unique, category)())
            mapping_anonymised = dict(zip(unique_value, faker_unique_list))
            mapping[col] = mapping_anonymised
            data[col] = data[col].map(mapping_anonymised)
    return data, mapping

In [ ]:
def anonymizer_multi(data_dict, anonymised_list):
    """
    Args:
        data_dict (dictionary of pandas.DataFrame)
            dictionary of dataframe to anonymise
        anonymised_list (list of dictionary)
            list of dictionary containing key as columns to anonymise & value as Faker category
    Returns:
        data (pandas.DataFrame)
            dataframe anonymised
    """
    mapping_multi = {}
    for table_name in data_dict:
        mapping_multi[table_name] = {}
    for anonymised_dict in anonymised_list:
        faker = Faker()
        faker.add_provider(CustomProvider())        
        for col in anonymised_dict['col']:
            print('col', col)
            unique_value = []
            faker_unique_list = []
            for table_name in anonymised_dict['table_name_list']:
                print('table', table_name)
                unique_value += list((data_dict[table_name][col]).unique())

                for i in range(len(unique_value)): 
                    faker_unique_list.append(getattr(faker.unique, anonymised_dict['category'])())
            
            mapping_anonymised = dict(zip(unique_value, faker_unique_list))
            for table_name in anonymised_dict['table_name_list']:
                mapping_multi[table_name][col] = mapping_anonymised
                data_dict[table_name][col] = data_dict[table_name][col].map(mapping_anonymised)
    return data_dict, mapping_multi 

In [ ]:
def anonymizer_iban_prepro(data, list_col):
    """
    Args:
        data (pandas.DataFrame)
        list_col (list of string)
            list of col to anonymize
    Returns:
        data anonymized (pandas.DataFrame)
    """
    for col in list_col:
        print('col', col)
        data[f'{col}_2_letters'] = data[col].str[:2]
        
    return data 

In [ ]:
def anonymizer_iban_post(data, list_col):
    """
    Args:
        data (pandas.DataFrame)
        list_col (list of string)
            list of col to anonymize
    Returns:
        data anonymized (pandas.DataFrame)
    """
    for col in list_col:
        print('col', col)
        data[f'{col}_new'] = data[col]
        data = data.drop(columns = [col])
        data[col] = data[f'{col}_2_letters'] + data[f'{col}_new'].str[2:]
        print('Dropping col')
        data = data.drop(columns = [f'{col}_2_letters', f'{col}_new'])
    return data 

In [ ]:
data_dict = {'test': ['a', 'b', 'c']}

In [ ]:
df = pd.DataFrame(data_dict)

In [ ]:
df

In [ ]:
list_company_name_account_anonym = ['test']

In [ ]:
anonymised_dict_account = [{'col': list_company_name_account_anonym, 'category': 'company'}]

In [ ]:
anonymised_dict_account

In [ ]:
anonymised_dict_account[0]['col']

In [ ]:
for anonymised_dict in anonymised_dict_account:
    print(anonymised_dict)
    for col in anonymised_dict['col']:
        print('col:', col)

In [ ]:
(df, anonymised_dict_account)

In [ ]:
mapping = {}
for anonymised_dict in anonymised_dict_account:
    print('anonymised_dict:', anonymised_dict)
    category = anonymised_dict['category']
    for col in anonymised_dict['col']:
        print('col:', col)
        faker_unique_list = []
        unique_value = list((df[col]).unique())
        faker = Faker()
        faker.add_provider(CustomProvider())
        for i in range(len(unique_value)): 
            print('category', category)
            faker_unique_list.append(getattr(faker.unique, category)())
        mapping_anonymised = dict(zip(unique_value, faker_unique_list))
        mapping[col] = mapping_anonymised
        df[col] = df[col].map(mapping_anonymised)


In [ ]:
for i in range(len(unique_value)): 
    print('category')
    faker_unique_list.append(getattr(faker.unique, 'company')())

## FakeGenerator Class

In [149]:
class FakeGenerator():
    def __init__(self, schema_list : str, seed : int = 123, custom_provider : CustomProvider = None):
        self.schema_list : list = schema_list
        self.seed = seed
        self.custom_provider : CustomProvider = custom_provider

    @property
    def set_seed(self, seed):
        self.seed = seed
    
    def generate_fake_columns(self, schema : dict, num_rows: int) -> list:
        """ Generate fake data for one column
        Args:
            schema :containing schema {column name, category, type, unique}
            num_rows : number of rows to generate
        Returns:
            list of values based on the faker category 
        """
        faker = Faker()
        Faker.seed(self.seed)
        faker.add_provider(self.custom_provider)
        return list(map(lambda x: getattr(faker, schema['category'])() , range(num_rows)))
        
    def generate_fake_data(self, num_rows) -> pd.DataFrame:
        """ Generate Fake data based on schema
        """
        data_columns = list(map(lambda schema, num_rows: self.generate_fake_columns(schema, num_rows), self.schema_list, [num_rows] * num_rows))
        data_columns = list(zip(*data_columns))
        fake_data = pd.DataFrame(data_columns, columns=[schema['column'] for schema in self.schema_list])
        return fake_data

In [ ]:
column_name = 'num_compte'
category_name = 'num_compte'
type = 'string'

In [35]:
num_rows = 5000
schema_list = [
{'column': 'name', 'category': 'name', 'type': type, 'unique': False},
{'column': 'address', 'category': 'address', 'type': type, 'unique': False},
{'column': 'job', 'category': 'job', 'type': type, 'unique': False},
{'column': 'company', 'category': 'company', 'type': type, 'unique': False},
]

In [112]:
def generate_fake_columns(schema):
    faker = Faker()
    Faker.seed(123)
    faker.add_provider(CustomProvider())
    return list(map(lambda x: getattr(faker, schema['category'])() , range(5)))

In [113]:
schema = {'column': 'name', 'category': 'name', 'type': type, 'unique': False}

In [114]:
generate_fake_columns(schema)

['Brandon Russell',
 'Steven Johnson',
 'Evelyn Christian',
 'George Cook',
 'Aaron Graham']

In [50]:
data = pd.DataFrame()
map(lambda x : generate_columns(data, x), schema_list)

In [57]:
data_columns = list(map(lambda schema: generate_fake_columns(schema), schema_list))
data_columns = list(zip(*data_columns))

In [59]:
data = pd.DataFrame(data_columns, columns=[schema['column'] for schema in schema_list])

In [60]:
data

,name,address,job,company
0,Jennifer Phelps,"735 Watkins Road\nPort Kristina, DE 87881",Geophysical data processor,"Tate, Schaefer and Curtis"
1,April Miller,"293 Robert River\nStaceyside, AR 09487",Travel agency manager,"Fox, Foster and Brown"
2,Donald Perry,"927 Ralph Shoal\nJohnsonborough, RI 65240",Agricultural consultant,Freeman-Reeves
3,Stephen Riley,"4362 Villanueva Parkways\nEast Roberttown, ND ...",Clinical research associate,Snyder-White
4,Diane Ayala,"0238 Taylor Rapids Apt. 741\nEast Briannafurt,...",Facilities manager,"Hunter, Sharp and Meadows"
5,Lisa Roberts,835 Booker Estate Suite 287\nLake Elizabethbor...,Equities trader,Smith-Price
6,Tina Velasquez,"18324 Goodwin Terrace\nHernandezchester, CA 51779",Solicitor,Mercer-Powers
7,Sarah Peterson,"70897 Wood Loop\nSouth Jamesshire, PW 67522",Trading standards officer,"Barajas, Chandler and Davis"
8,Tina Gonzalez,"44971 Kenneth Mount\nCarlsonshire, PA 75377",Transport planner,Washington and Sons
9,Sarah Davis,"34832 Wade Passage Suite 699\nHillport, WV 35797",Hydrogeologist,Joseph and Sons


In [133]:
data = pd.DataFrame()
for schema in schema_list:
    data[schema['column']] = faker_list

In [134]:
data

,name,address,job,company
0,Shelton Group,Shelton Group,Shelton Group,Shelton Group
1,Wilkins and Sons,Wilkins and Sons,Wilkins and Sons,Wilkins and Sons
2,Erickson-Valdez,Erickson-Valdez,Erickson-Valdez,Erickson-Valdez
3,Dickerson-Small,Dickerson-Small,Dickerson-Small,Dickerson-Small
4,Taylor-Barron,Taylor-Barron,Taylor-Barron,Taylor-Barron
5,Rodgers-Brooks,Rodgers-Brooks,Rodgers-Brooks,Rodgers-Brooks
6,Clark-Griffith,Clark-Griffith,Clark-Griffith,Clark-Griffith
7,Williams and Sons,Williams and Sons,Williams and Sons,Williams and Sons
8,"Pace, Rodriguez and Lee","Pace, Rodriguez and Lee","Pace, Rodriguez and Lee","Pace, Rodriguez and Lee"
9,Jordan PLC,Jordan PLC,Jordan PLC,Jordan PLC


In [147]:
num_rows = 5
seed = 123

In [150]:
start = time.time()
test_data = FakeGenerator(schema_list, seed, CustomProvider()).generate_fake_data(num_rows)
stop = time.time()
print(f'time generation : {stop - start}')

time generation : 0.055381059646606445


In [151]:
test_data.shape

(5, 4)

In [152]:
test_data

,name,address,job,company
0,Brandon Russell,"4106 Peterson Center\nEast Matthew, MA 92472",Associate Professor,Adams Group
1,Steven Johnson,"1961 Leslie Mission\nNorth Christopherberg, DC...","Geologist, wellsite",Rodriguez-Santana
2,Evelyn Christian,"747 Kayla Junctions Suite 398\nAlexanderhaven,...",Chartered loss adjuster,Gray PLC
3,George Cook,"658 Melanie Rue Apt. 174\nWest Matthew, MO 10107",Orthoptist,Peterson-Williams
4,Aaron Graham,"503 Marie Lakes Apt. 536\nLake Keithfort, WV 8...",Geographical information systems officer,Rogers LLC


In [153]:
test_data.to_dict()

{'name': {0: 'Brandon Russell',
  1: 'Steven Johnson',
  2: 'Evelyn Christian',
  3: 'George Cook',
  4: 'Aaron Graham'},
 'address': {0: '4106 Peterson Center\nEast Matthew, MA 92472',
  1: '1961 Leslie Mission\nNorth Christopherberg, DC 03284',
  2: '747 Kayla Junctions Suite 398\nAlexanderhaven, NE 90263',
  3: '658 Melanie Rue Apt. 174\nWest Matthew, MO 10107',
  4: '503 Marie Lakes Apt. 536\nLake Keithfort, WV 83075'},
 'job': {0: 'Associate Professor',
  1: 'Geologist, wellsite',
  2: 'Chartered loss adjuster',
  3: 'Orthoptist',
  4: 'Geographical information systems officer'},
 'company': {0: 'Adams Group',
  1: 'Rodriguez-Santana',
  2: 'Gray PLC',
  3: 'Peterson-Williams',
  4: 'Rogers LLC'}}

In [128]:
[10] * 10

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

### List of Category Faker 
- Faire une liste des categories existantes dans Faker 

**address**
- address
- building_number
- city
- city_suffix
- country
- country_code
- current_country
- current_country_code()
- postcode
- street_address
- street_name
- street_suffix

**bank**
- aba
- bank_country
- bban
- iban
- swift
- swift8
- swift11

**company**
- bs
- catch_phrase
- company
- company_suffix

**currency:**
- cryptocurrency
- cryptocurrency_code
- cryptocurrency_name
- currency_code
- currency_name
- currency_symbol
- pricetag

**phone_number:**
- country_calling_code
- msisdn
- phone_number

**person:**
- first_name
- first_name_female
- first_name_male
- language_name
- last_name
- name
- suffix
- job

**geo:**
- coordinate
- latitude
- latlng
- longitude

**credit_card_number:**
- credit_card_expire
- credit_card_full
- credit_card_number
- credit_card_provider
- credit_card_security_code

### Unique Value 

In [62]:
getattr(faker.unique, 'name')()

'Miranda King'